In [ ]:
import os
import pandas as pd
import re

def extract_metadata_from_folder(folder_path):
    summary = []

    # Ustalona kolejność jednostek interwału
    def oczyszczona_nazwa(nazwa: str) -> str:
        # Usuń końcówkę 'USDT' jeśli występuje
        nazwa = re.sub(r'USDT$', '', nazwa)
        # Usuń wszystko po kropce, np. '.OANDA'
        nazwa = nazwa.split('.')[0]
        return nazwa

    def extract_interval_key(name):
        match = re.search(r'(\d+)([mhdDwWM])', name)
        if not match:
            return (float('inf'), float('inf'))
        
        num = int(match.group(1))
        unit = match.group(2)

        # Małe m = 1, d = 2, w = 3, duże M = 4
        if unit == 'm':
            order = 1
        elif unit == 'h':
            order = 2
        elif unit == 'd':
            order = 3
        elif unit in ['w', 'W']:
            order = 4
        elif unit == 'M':
            order = 5
        else:
            order = float('inf')

        return (order, num)

    for root, dirs, files in os.walk(folder_path):
        if not files:
            continue

        # Tylko nazwy plików z interwałami
        intervals = [os.path.splitext(f)[0] for f in files if re.search(r'\d+[mhdDwWM]', f)]
        if not intervals:
            continue

        # Sortuj interwały według ustalonego klucza
        sorted_intervals = sorted(intervals, key=extract_interval_key)

        # Pliki CSV z interwałami
        csv_files = [f for f in files if f.endswith('.csv') and re.search(r'\d+[mhdDwWM]', f)]
        sorted_csv_files = sorted(csv_files, key=lambda f: extract_interval_key(os.path.splitext(f)[0]))

        if not sorted_csv_files:
            continue

        min_interval_file = sorted_csv_files[0]
        full_path = os.path.join(root, min_interval_file)

        # Ustal nazwę interwału (np. 5m, 1d)
        match = re.search(r'\d+[mhdDwWM]', min_interval_file)
        min_interval = match.group(0) if match else None

        try:
            df = pd.read_csv(full_path)
        except Exception as e:
            print(f"⚠️ Błąd wczytywania pliku {min_interval_file}: {e}")
            continue

        if 'timestamp' in df.columns:
            timestamps = pd.to_datetime(df['timestamp'])
        elif 'date' in df.columns:
            timestamps = pd.to_datetime(df['date'])
        else:
            continue

        poczatek = timestamps.min().date()
        koniec = timestamps.max().date()
        

        nazwa = os.path.basename(root)
        category=""
        category = "CRYPTO NEW" if nazwa.endswith("USDT") else None
        ticker = oczyszczona_nazwa(nazwa)
        summary.append({
            'Ticker': ticker,
            'Początek': poczatek,
            'Koniec': koniec,
            'Min. interwał': min_interval,
            'Plik': nazwa,
            "Category": category,
            'Intervals': sorted_intervals

        })

    return pd.DataFrame(summary)


# 📁 Ścieżka do folderu z danymi
katalog = "row_data"

# 🔁 Tworzymy podsumowanie
tabela = extract_metadata_from_folder(katalog)

# 💾 Zapisz do Excela
tabela.to_excel("Market_data.xlsx", index=False)

print("Zapisano do Market_data.xlsx")